# Meshing

## Inviscid Wedge
![Inviscid Wedge](../assets/inviscid_wedge.png)

https://su2code.github.io/tutorials/Inviscid_Wedge/

In [2]:
import cadquery as cq
from ezmesh import GeometryQL, Split


with GeometryQL() as geo:
    geo = (
        geo
        .load(  
            cq.Workplane("XY")
            .polyline([(0, 1), (1.5, 1), (1.5, 0.2), (0.5, 0), (0, 0)])
            .close(),
            splits= lambda workplane: [
                Split.from_lines(workplane, ((0.5,0), (0.5,1)))
            ]
        )

        .setTransfiniteAuto(100)

        .fromTagged([f"edge/{i}" for i in [1,2,4,5,6,7]], "edge")
        .addPhysicalGroup(["inlet", "lower", "upper", "lower", "outlet", "upper"])
        .end()

        .recombine()


        .generate()
        .show("mesh")
    )


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.000363042s, CPU 0.000584s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 50%] Meshing surface 2 (Transfinite)
Info    : Done meshing 2D (Wall 0.00172308s, CPU 0.001628s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 5.08397e-06s, CPU 5e-06s)
Info    : 20301 nodes 20706 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,lower
,inlet
,upper
,outlet
,Zone 0
